In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
data=pd.read_csv("AudioBookNew.csv")
data

,ID,BookLength (mins) Overall,BookLength (mins)_avg,Price Overall,Price Avg,Review,Review 10/10,Minutes Listened,Completion,Support Requests,Last visited minus Purchased date,Targets
0,994,1620.0,1620,19.73,19.73,1,10.00,0.99,1603.8,5,92,0
1,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
2,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
3,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
4,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14079,28220,1620.0,1620,5.33,5.33,1,9.00,0.61,988.2,0,4,0
14080,28671,1080.0,1080,6.55,6.55,1,6.00,0.29,313.2,0,29,0
14081,31134,2160.0,2160,6.14,6.14,0,8.91,0.00,0.0,0,0,0
14082,32832,1620.0,1620,5.33,5.33,1,8.00,0.38,615.6,0,90,0


In [4]:
raw_csv_data=np.loadtxt("Audiobooks_data.csv", delimiter=",")
unscaled_inputs_all=raw_csv_data[:,1:-1]
unscaled_inputs_all


array([[1620.  , 1620.  ,   19.73, ..., 1603.8 ,    5.  ,   92.  ],
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,    0.  ],
       [2160.  , 2160.  ,    5.33, ...,    0.  ,    0.  ,  388.  ],
       ...,
       [2160.  , 2160.  ,    6.14, ...,    0.  ,    0.  ,    0.  ],
       [1620.  , 1620.  ,    5.33, ...,  615.6 ,    0.  ,   90.  ],
       [1674.  , 3348.  ,    5.33, ...,    0.  ,    0.  ,    0.  ]])

In [5]:
targets_all=raw_csv_data[:,-1]

# BALANCE THE DATASET

In [6]:
no_one_target=np.sum(targets_all)
#targets_all.shape gives no of rows so
zero_target_counter=0
indices_to_remove=[]
for i in range(targets_all.shape[0]):
  if targets_all[i]==0:
    zero_target_counter+=1
    if zero_target_counter>no_one_target:
      indices_to_remove.append(i)

unscaled_inputs_equals_priors=np.delete(unscaled_inputs_all,indices_to_remove, axis=0)
targets_equal_priors=np.delete(targets_all, indices_to_remove, axis=0)



# Standardize Inputs

In [7]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equals_priors)

# Shuffle Data

In [8]:
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_equal_priors[shuffled_indices]


# SPLIT


In [11]:
sample_count=shuffled_inputs.shape[0]
train_samples_count=int(0.8*sample_count)
validate_samples_count=int(0.1*sample_count)
test_samples_count=sample_count-train_samples_count-validate_samples_count

train_inputs=shuffled_inputs[:train_samples_count]
train_targets=shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+validate_samples_count]
validation_targets=shuffled_targets[train_samples_count:train_samples_count+validate_samples_count]

test_inputs=shuffled_inputs[train_samples_count+validate_samples_count:]
test_targets=shuffled_targets[train_samples_count+validate_samples_count:]

print(np.sum(train_targets), train_samples_count,np.sum(train_targets)/train_samples_count )

print(np.sum(validation_targets), validate_samples_count,np.sum(validation_targets)/validate_samples_count )

print(np.sum(test_targets), test_samples_count,np.sum(test_targets)/test_samples_count )

1785.0 3579 0.49874266554903607
221.0 447 0.49440715883668906
231.0 448 0.515625


this 0.5 signifies all datasets have 50 50 cases of both scenarios XD

# Change to NPZ


In [13]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

